Just trying stuff out
====

In [ ]:
"""
Read in some WT jaws
"""

import tifffile
from tqdm.notebook import tqdm

from fishjaw.inference import read
from fishjaw.util import files

output_dir = files.script_out_dir() / "jaw_segmentations"

img_dir = output_dir / "imgs"
mask_dir = output_dir / "masks"

metadata_df = read.mastersheet()

wt_mdata_df = metadata_df[metadata_df["name"].str.contains("wt")]

imgs = []
masks = []

for n in tqdm(wt_mdata_df.index):
    try:
        new_img = tifffile.imread(img_dir / f"ak_{n}.tif")
        new_mask = tifffile.imread(mask_dir / f"ak_{n}.tif")
    except FileNotFoundError:
        print(f"{n} doesn't exist")

    imgs.append(new_img)
    masks.append(new_mask)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# hacky
from scripts.pipeline.plot_3d import _calculate_point_size

fig = plt.figure(figsize=(16, 6))

n = 6
axes = [fig.add_subplot(1, n, i + 1, projection="3d") for i in range(n)]

plot_kw = {
    "marker": "s",
    "cmap": "inferno",
    "vmin": 0,
    "vmax": 2**16,
    "s": _calculate_point_size(axes[0], imgs[0].shape),
}

for img, mask, axis in zip(imgs, masks, axes, strict=False):
    co_ords = np.argwhere(mask)
    greyscale_vals = img[co_ords[:, 0], co_ords[:, 1], co_ords[:, 2]]

    axis.scatter(
        co_ords[:, 0], co_ords[:, 1], co_ords[:, 2], c=greyscale_vals, **plot_kw
    )

    axis.view_init(elev=45, azim=-90, roll=-140)
    axis.set_axis_off()